In [1]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score as auc

In [2]:
from mriqc_learn.datasets import load_dataset
from mriqc_learn.models import preprocess as pp
from mriqc_learn.models.production import init_pipeline
from mriqc_learn.model_selection import split

## Load some data
We first load the ABIDE dataset, one of the default datasets distributed with MRIQC-learn

In [3]:
(train_x, train_y), (_, _) = load_dataset(split_strategy="none")
train_x["site"] = train_y.site

Let's pick the ratings from "rater_3" and binarize the three categories into only two.
We can also see that the dataset is unbalanced.

In [4]:
train_y = train_y[["rater_3"]].values.squeeze().astype(int)
print(f"Discard={100 * (train_y == -1).sum() / len(train_y)}")
print(f"Doubtful={100 * (train_y == 0).sum() / len(train_y)}")
print(f"Accept={100 * (train_y == 1).sum() / len(train_y)}")
train_y[train_y < 1] = 0

Discard=14.168937329700272
Doubtful=1.5440508628519527
Accept=84.28701180744777


Let's print out a pretty view of the data table:

In [5]:
train_x

cjv       cnr       efc         fber  fwhm_avg    fwhm_x    fwhm_y  \
0     0.383747  3.259968  0.609668   181.619858  3.944888  3.959924  4.039157   
1     0.574080  2.279440  0.606361   172.500031  3.992397  3.877495  4.173095   
2     0.314944  3.998569  0.577123   273.688171  4.016382  4.066009  4.092888   
3     0.418505  3.050534  0.571343   237.531143  3.601741  3.629409  3.627568   
4     0.286560  4.214082  0.550083   427.042389  3.808350  3.839143  3.841085   
...        ...       ...       ...          ...       ...       ...       ...   
1096  0.428731  3.030323  0.789654  2519.999512  3.176760  3.166740  3.359990   
1097  0.610845  2.155928  0.800116  1769.720093  3.209497  3.164760  3.381280   
1098  0.461773  2.794299  0.789859  2248.858398  3.149920  3.112220  3.326700   
1099  0.457718  2.862913  0.706924   114.865364  3.486750  3.421200  3.881950   
1100  0.419648  3.129415  0.783152   116.484947  3.579490  3.538990  3.991800   

        fwhm_z  icvs_csf   icvs_gm  ...  summary_wm_median  summary_wm_n  \
0     3.835584  0.199774  0.449138  ...        1000.013428      189965.0   
1     3.926602  0.203301  0.429628  ...        1000.033569      187992.0   
2     3.890248  0.201591  0.446495  ...        1000.015198      188213.0   
3     3.548246  0.190612  0.468255  ...        1000.005981      146722.0   
4     3.744823  0.162421  0.505201  ...        1000.004150      162584.0   
...        ...       ...       ...  ...                ...           ...   
1096  3.003550  0.169851  0.424819  ...        1000.034668      241117.0   
1097  3.082450  0.170732  0.405536  ...        1000.039429      251136.0   
1098  3.010840  0.165501  0.441190  ...        1000.036438      209298.0   
1099  3.157100  0.209701  0.381839  ...         999.990356      234957.0   
1100  3.207680  0.215531  0.402868  ...        1000.005981      244797.0   

      summary_wm_p05  summary_wm_p95  summary_wm_stdv  tpm_overlap_csf  \
0         908.938904     1079.413428        51.778980         0.225944   
1         901.788293     1120.833569        67.136932         0.223374   
2         913.847803     1067.003662        46.623932         0.233414   
3         872.409717     1083.139264        63.131420         0.227282   
4         900.433481     1069.912750        50.874363         0.195150   
...              ...             ...              ...              ...   
1096      902.529590     1088.244409        56.683868         0.162535   
1097      903.951080     1093.323273        57.789230         0.193376   
1098      891.934216     1093.973322        61.108639         0.198508   
1099      904.907922     1101.429980        60.045422         0.235618   
1100      904.572144     1095.835376        58.381458         0.227491   

      tpm_overlap_gm  tpm_overlap_wm    wm2max     site  
0           0.525072        0.540801  0.540213     PITT  
1           0.521399        0.560238  0.571425     PITT  
2           0.531020        0.556496  0.612655     PITT  
3           0.528115        0.526254  0.600312     PITT  
4           0.543591        0.531606  0.603308     PITT  
...              ...             ...       ...      ...  
1096        0.476992        0.536843  0.537140      SBL  
1097        0.465232        0.545695  0.564010      SBL  
1098        0.497137        0.523571  0.564865      SBL  
1099        0.477310        0.563352  0.534626  MAX_MUN  
1100        0.492317        0.549036  0.573174  MAX_MUN  

[1101 rows x 69 columns]

## Cross-validation of the default classifier
Let's cross-validate the performance of our classifier using a Leave-one-site-out strategy.

In [6]:
# Define a splitting strategy
outer_cv = split.LeavePSitesOut(1, robust=True)

We can now feed the model into the cross-validation loop:

In [7]:
cv_score = cross_val_score(
    init_pipeline(),
    X=train_x,
    y=train_y,
    cv=outer_cv,
    scoring="roc_auc",
    n_jobs=16,
)

After one or two minutes, the scores have been caculated for each of the 14 folds our splitter created.
The average performance is AUC=0.885.

In [8]:
print(cv_score)
cv_score.mean()

[0.89230769 0.78085106 0.88235294 0.91823899 0.90697674 0.77705263
 1.         0.94350282 1.         0.98174603 1.         0.84433962
 0.32432432 0.96938776]


0.8729343303901641

In [9]:
custom_cv_score = {}
for train_index, (site, test_index) in outer_cv.split(train_x, y=train_y, return_key=True):
    # Validate on test fold
    print(f"Validating on left-out site ({site})...")
    model_split = init_pipeline()
    model_split = model_split.fit(train_x.iloc[train_index], train_y[train_index])
    custom_cv_score[site] = auc(train_y[test_index], model_split.predict(train_x.iloc[test_index]))

Validating on left-out site (OLIN)...
Validating on left-out site (MAX_MUN)...
Validating on left-out site (SDSU)...
Validating on left-out site (YALE)...
Validating on left-out site (TRINITY)...
Validating on left-out site (UM)...
Validating on left-out site (LEUVEN)...
Validating on left-out site (NYU)...
Validating on left-out site (KKI)...
Validating on left-out site (UCLA)...
Validating on left-out site (SBL)...
Validating on left-out site (PITT)...
Validating on left-out site (CALTECH)...
Validating on left-out site (USM)...


In [10]:
print(custom_cv_score)
np.mean(list(custom_cv_score.values()))

{'OLIN': 0.7, 'MAX_MUN': 0.6074468085106384, 'SDSU': 0.6176470588235294, 'YALE': 0.6194968553459119, 'TRINITY': 0.8333333333333334, 'UM': 0.52, 'LEUVEN': 0.9841269841269842, 'NYU': 0.6400322841000807, 'KKI': 1.0, 'UCLA': 0.7880952380952381, 'SBL': 0.9827586206896552, 'PITT': 0.625, 'CALTECH': 0.5, 'USM': 0.8078231292517007}


0.7304114508769336

We now train the model on all available training data:

In [11]:
model = init_pipeline().fit(
    X=train_x,
    y=train_y,
)    

We can easily see the effects of overfitting by evaluating the classifier on the same folds we used for cross-validation.

In [12]:
overfit_cv_score = {}
for train_index, (site, test_index) in outer_cv.split(train_x, y=train_y, return_key=True):
    print(f"Validating on left-out site ({site})...")
    overfit_cv_score[site] = auc(train_y[test_index], model.predict(train_x.iloc[test_index]))

Validating on left-out site (OLIN)...
Validating on left-out site (MAX_MUN)...
Validating on left-out site (SDSU)...
Validating on left-out site (YALE)...
Validating on left-out site (TRINITY)...
Validating on left-out site (UM)...
Validating on left-out site (LEUVEN)...
Validating on left-out site (NYU)...
Validating on left-out site (KKI)...
Validating on left-out site (UCLA)...
Validating on left-out site (SBL)...
Validating on left-out site (PITT)...
Validating on left-out site (CALTECH)...
Validating on left-out site (USM)...


In [13]:
print([overfit_cv_score[s] - custom_cv_score[s] for s in overfit_cv_score.keys()])

[0.10000000000000009, 0.021276595744680882, 0.05882352941176472, 0.3616352201257862, 0.0, 0.13, 0.015873015873015817, 0.07425343018563357, 0.0, 0.0726190476190477, 0.0, 0.0, 0.0, 0.1768707482993196]


In [14]:
from sklearn.metrics import classification_report

print(classification_report(train_y, model.predict(train_x)))

              precision    recall  f1-score   support

           0       0.86      0.38      0.53       173
           1       0.90      0.99      0.94       928

    accuracy                           0.89      1101
   macro avg       0.88      0.68      0.73      1101
weighted avg       0.89      0.89      0.87      1101



## Evaluating on held-out dataset
We first load the held-out dataset in, and evaluate:

In [15]:
(test_x, test_y), (_, _) = load_dataset("ds030", split_strategy="none")
test_x["site"] = test_y.site
test_x

cjv       cnr       efc         fber  fwhm_avg    fwhm_x   fwhm_y  \
0    0.550186  2.459577  0.507058  1065.732178  3.480117  3.507830  3.70268   
1    0.456006  2.921997  0.561604   799.381470  3.321400  3.347790  3.50000   
2    0.445959  2.908593  0.549342  1123.041870  3.137148  3.138153  3.35693   
3    0.767179  1.766171  0.568210   630.778992  3.339373  3.364539  3.45910   
4    0.539929  2.473583  0.541802   937.298462  3.108547  3.133720  3.26695   
..        ...       ...       ...          ...       ...       ...      ...   
260  0.433127  3.111099  0.562418   886.391785  3.352552  3.442397  3.51022   
261  0.455297  2.946393  0.494483   985.328186  3.015977  3.059440  3.12296   
262  0.428027  3.056015  0.544575   899.129028  3.158103  3.193888  3.29329   
263  0.456340  2.968104  0.531945   963.673828  3.203233  3.230830  3.41218   
264  0.513471  2.557568  0.543417   777.718445  3.317640  3.360951  3.48349   

      fwhm_z  icvs_csf   icvs_gm  ...  summary_wm_median  summary_wm_n  \
0    3.22984  0.243019  0.395910  ...        1000.057861      183285.0   
1    3.11641  0.175314  0.463107  ...        1000.007141      193080.0   
2    2.91636  0.174694  0.411513  ...         999.994690      205413.0   
3    3.19448  0.209167  0.389043  ...        1000.085632      255944.0   
4    2.92497  0.198797  0.442200  ...        1000.033508      170103.0   
..       ...       ...       ...  ...                ...           ...   
260  3.10504  0.198912  0.418726  ...         999.999268      236049.0   
261  2.86553  0.207224  0.421287  ...        1000.008179      152038.0   
262  2.98713  0.188024  0.418417  ...         999.986328      200700.0   
263  2.96669  0.184849  0.435990  ...        1000.027588      162744.0   
264  3.10848  0.198040  0.401341  ...        1000.022705      252528.0   

     summary_wm_p05  summary_wm_p95  summary_wm_stdv  tpm_overlap_csf  \
0        883.837659     1149.885156        81.690285         0.215809   
1        892.560648     1105.589551        64.899048         0.233023   
2        903.285217     1099.551831        59.837898         0.230147   
3        879.354327     1157.930383        85.289146         0.228836   
4        894.828833     1112.822083        66.272682         0.201488   
..              ...             ...              ...              ...   
260      896.524268     1102.766406        62.811314         0.208632   
261      905.155991     1092.722668        57.217701         0.172909   
262      904.591296     1087.129657        55.752438         0.222306   
263      898.410342     1099.041461        61.291889         0.214909   
264      904.592081     1101.841998        60.304161         0.211870   

     tpm_overlap_gm  tpm_overlap_wm    wm2max  site  
0          0.476166        0.560368  0.677318   BMC  
1          0.535007        0.564211  0.649004   BMC  
2          0.492765        0.577273  0.611447   BMC  
3          0.474763        0.554115  0.630658   BMC  
4          0.497095        0.538423  0.661969   BMC  
..              ...             ...       ...   ...  
260        0.502756        0.565940  0.651403   CCN  
261        0.490781        0.573420  0.667625   CCN  
262        0.495817        0.566268  0.669586   CCN  
263        0.495231        0.545337  0.612139   CCN  
264        0.479458        0.576943  0.637148   CCN  

[265 rows x 69 columns]

In [16]:
has_ghost = test_y.has_ghost.values.astype(bool)
test_y = test_y[["rater_1"]].values.squeeze().astype(int)
print(f"Discard={100 * (test_y == -1).sum() / len(test_y)}")
print(f"Doubtful={100 * (test_y == 0).sum() / len(test_y)}")
print(f"Accept={100 * (test_y == 1).sum() / len(test_y)}")
test_y[test_y < 1] = 0

Discard=28.30188679245283
Doubtful=54.716981132075475
Accept=16.9811320754717


In [17]:
auc(test_y, model.predict(test_x))

0.5363636363636364

In [18]:
auc(test_y[~has_ghost], model.predict(test_x[~has_ghost]))

0.5319767441860466

In [19]:
print(classification_report(test_y, model.predict(test_x)))

              precision    recall  f1-score   support

           0       1.00      0.07      0.14       220
           1       0.18      1.00      0.31        45

    accuracy                           0.23       265
   macro avg       0.59      0.54      0.22       265
weighted avg       0.86      0.23      0.16       265



In [20]:
print(classification_report(test_y[~has_ghost], model.predict(test_x[~has_ghost])))

              precision    recall  f1-score   support

           0       1.00      0.06      0.12       172
           1       0.22      1.00      0.36        45

    accuracy                           0.26       217
   macro avg       0.61      0.53      0.24       217
weighted avg       0.84      0.26      0.17       217



## Nested cross-validation

In [ ]:
p_grid = [{
    "scale__unit_variance": [True, False],
    "scale__with_centering": [True, False],
    "site_pred__disable": [False, True],
    "winnow__disable": [False, True],
    "svc__kernel": ["rbf"],
    "svc__C": [10],
    "svc__gamma": [0.1],
}]

In [ ]:
# Nested CV with parameter optimization
inner_cv = split.LeavePSitesOut(1, robust=True)
inner_cv.get_n_splits(X=train_x, y=train_y)

clf = GridSearchCV(
    estimator=pipe,
    param_grid=p_grid,
    cv=inner_cv,
    n_jobs=30,
    scoring="roc_auc",
)
# clf.fit(train_x, y=train_y)

In [ ]:
nested_score = cross_val_score(
    clf,
    X=train_x,
    y=train_y,
    cv=outer_cv,
    scoring="roc_auc",
    verbose=10,
    n_jobs=16,
)
nested_score.mean()

In [ ]:
clf.cv_results_

In [ ]:
clf.best_params_